In [1]:
import numpy as np
import math
from scipy.optimize import linear_sum_assignment
from scipy.optimize import minimize_scalar
from sklearn.utils import check_array
from sklearn.utils import column_or_1d
from skp import SinkhornKnopp  # .
from joblib import Parallel, delayed

In [29]:
from par_gmp import GraphMatch

In [4]:
from graspy.plot import heatmap
from graspy.simulations import er_corr, sbm, sbm_corr
import random
import time
import datetime as dt
from graspy.match import GraphMatch as GMP

In [5]:
np.random.seed(8888)
directed = False
loops = False
n_per_block = 50
n_blocks = 3
block_members = np.array(n_blocks * [n_per_block])
n_verts = block_members.sum()
rho = 0.5
block_probs = np.array([[0.7, 0.3, 0.4], [0.3, 0.7, 0.3], [0.4, 0.3, 0.7]])

In [20]:
# run GMP in serial
A1, A2 = sbm_corr(block_members, block_probs, rho, directed=directed, loops=loops)
# shuffle for testing
node_shuffle_input = np.random.permutation(n_verts)
A2_shuffle = A2[np.ix_(node_shuffle_input, node_shuffle_input)]
node_unshuffle_input = np.array(range(n_verts))
node_unshuffle_input[node_shuffle_input] = np.array(range(n_verts))

n_init = 50  # parameter for GMP
start = dt.datetime.now()

sgm = GMP(n_init=n_init, init_method="rand")
sgm = sgm.fit(A1, A2_shuffle)

A2_unshuffle = A2_shuffle[np.ix_(sgm.perm_inds_, sgm.perm_inds_)]
match_ratio = 1 - (
    np.count_nonzero(abs(sgm.perm_inds_ - node_unshuffle_input)) / n_verts
)
print(f"{dt.datetime.now() - start} elapsed for serial")

0:00:27.678517 elapsed for serial


In [21]:
print("Match Ratio (serial) ", match_ratio)
print("Optimal objective (serial) ", sgm.score_)

Match Ratio (serial)  0.033333333333333326
Optimal objective (serial)  6448.0


In [50]:
# run GMP in parallel
A1, A2 = sbm_corr(block_members, block_probs, rho, directed=directed, loops=loops)
# shuffle for testing
node_shuffle_input = np.random.permutation(n_verts)
A2_shuffle = A2[np.ix_(node_shuffle_input, node_shuffle_input)]
node_unshuffle_input = np.array(range(n_verts))
node_unshuffle_input[node_shuffle_input] = np.array(range(n_verts))

n_init = 50
start = dt.datetime.now()
np.random.seed(12345)
sgm_par = GraphMatch(n_init=n_init, init_method="rand")
sgm_par = sgm_par.fit(A1, A2_shuffle)

A2_unshuffle = A2_shuffle[np.ix_(sgm_par.perm_inds_, sgm_par.perm_inds_)]
match_ratio = 1 - (np.count_nonzero(abs(sgm_par.perm_inds_ - node_unshuffle_input)) / n_verts)

print(f"{dt.datetime.now() - start} elapsed for parallel")

0:00:17.591734 elapsed for parallel


In [51]:
print("Match Ratio (parallel) ", match_ratio)
print("Optimal objective (parallel) ", sgm_par.score_)

Match Ratio (parallel)  0.00666666666666671
Optimal objective (parallel)  6222.0


In [52]:
# run GMP in serial
n_init = 120  # parameter for GMP
start = dt.datetime.now()

sgm = GMP(n_init=n_init, init_method="rand")
sgm = sgm.fit(A1, A2_shuffle)

A2_unshuffle = A2_shuffle[np.ix_(sgm_par.perm_inds_, sgm_par.perm_inds_)]
match_ratio = 1 - (np.count_nonzero(abs(sgm_par.perm_inds_ - node_unshuffle_input)) / n_verts)

print(f"{dt.datetime.now() - start} elapsed for serial")

0:01:07.374317 elapsed for serial


In [12]:
print("Match Ratio (serial) ", match_ratio)
print("Optimal objective (serial) ", sgm.score_)

Match Ratio (serial)  0.013333333333333308
Optimal objective (serial)  6522.0


In [59]:
# run GMP in parallel
A1, A2 = sbm_corr(block_members, block_probs, rho, directed=directed, loops=loops)
# shuffle for testing
node_shuffle_input = np.random.permutation(n_verts)
A2_shuffle = A2[np.ix_(node_shuffle_input, node_shuffle_input)]
node_unshuffle_input = np.array(range(n_verts))
node_unshuffle_input[node_shuffle_input] = np.array(range(n_verts))

n_init = 120
start = dt.datetime.now()
np.random.seed(12345)
sgm_par = GraphMatch(n_init=n_init, init_method="rand")
sgm_par = sgm_par.fit(A1, A2_shuffle)

A2_unshuffle = A2_shuffle[np.ix_(sgm_par.perm_inds_, sgm_par.perm_inds_)]
match_ratio = 1 - (np.count_nonzero(abs(sgm_par.perm_inds_ - node_unshuffle_input)) / n_verts)

print(f"{dt.datetime.now() - start} elapsed for parallel")

0:00:38.247793 elapsed for parallel


In [60]:
print("Match Ratio (parallel) ", match_ratio)
print("Optimal objective (parallel) ", sgm_par.score_)

Match Ratio (parallel)  0.0
Optimal objective (parallel)  6504.0


In [55]:
# run GMP in parallel
A1, A2 = sbm_corr(block_members, block_probs, rho, directed=directed, loops=loops)
# shuffle for testing
node_shuffle_input = np.random.permutation(n_verts)
A2_shuffle = A2[np.ix_(node_shuffle_input, node_shuffle_input)]
node_unshuffle_input = np.array(range(n_verts))
node_unshuffle_input[node_shuffle_input] = np.array(range(n_verts))

n_init = 200
start = dt.datetime.now()
np.random.seed(12345)
sgm_par = GraphMatch(n_init=n_init, init_method="rand")
sgm_par = sgm_par.fit(A1, A2_shuffle)

A2_unshuffle = A2_shuffle[np.ix_(sgm_par.perm_inds_, sgm_par.perm_inds_)]
match_ratio = 1 - (np.count_nonzero(abs(sgm_par.perm_inds_ - node_unshuffle_input)) / n_verts)

print(f"{dt.datetime.now() - start} elapsed for parallel")

0:01:02.754261 elapsed for parallel


In [56]:
print("Match Ratio (parallel) ", match_ratio)
print("Optimal objective (parallel) ", sgm_par.score_)

Match Ratio (parallel)  0.07999999999999996
Optimal objective (parallel)  6540.0
